저상버스의 시간표를 저장하는 DB

In [2]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy.sql import select

In [3]:
engine = create_engine("sqlite:///lowbus_timetable.db", echo=False)
metadata = MetaData()

Timetable = Table('Timetable', metadata, 
              Column('_id', Integer, primary_key =True, autoincrement=True ),
              Column('routeId', Integer, nullable = False),
              Column('date', String(8), nullable = False), 
              Column('arrivalTm',String(6)),
              Column('posX',String), 
              Column('posY',String))  

metadata.create_all(engine) 
conn = engine.connect()

버스위치api를 반복 호출하면서 현재 운행중인 저상버스가 정류소에 도착하는 시간을 DB에 저장
- 호출했을 때의 위치정보 제공 시간이 이전 호출했을 때의 제공 시간과 다르면서 정류장에 도착해있을 때(stopflag=1) 저장
- 여러 서비스키를 사용하여 하나의 키의 traffic을 모두 사용했을 때 다른 서비스 키 사용


In [4]:
import requests
from bs4 import BeautifulSoup
import time

In [5]:
serviceKey = [
'17fE4NH591MbdnqzN321LiGIFYgJ6XG3LEwBDNDeVYFQm6B9cG24yEdGLjpUa0zu7L1uJagT1fAlxXl9YlSd7Q%3D%3D',
'xsYCorXQLS6GeIHlSblN8cDUql3lya%2FhmRun3ZWAg4uwyPSPZX1ymJcPzt9yDNco1RdLO0Q28d542JfeI%2FoGRQ%3D%3D',
'pluGlBNSzwcQD9L6%2Fq%2B37%2F7ZZOYcjonWJ7eeqjUPy8dmhx9NxyTeEHPsmaXOekWlvImkztRMjZDI4Asp2%2BXdyA%3D%3D',
'ydnnpHpT6l2Y4DExg3mptI1GVHORb0OrGUz%2FlWFNg1rTa5OdqRKmHtfct8SF%2B84E84ZNRev03220ZCpElCzKGQ%3D%3D',
'fAVoRthJ7w6qLml%2Bs%2FVn2z6uXx5wba3vEIUZtl8S73v30WXT0RoQTJmOyDa6BhmRCxO3vaI0UJkOwZdIrosY7g%3D%3D',
'7tEdncbTNM%2B53QBBwElYZLgu1wPpxc3oZn%2FMIceVEB%2Fn3MPrYbIoDPlcsYLPAcCnNQpSTMq7A%2FMM%2BSycwtDm8Q%3D%3D',
'WKji4TSJk6O7Q8LBRLJma0%2BpGuGuGwC5%2FHYgacryM4AzLtHo8zccch60szLmam7YBLzPPPfSKBfVu9hNXFEYGA%3D%3D',
'eFOmgSlNbqagW9tpdkIXv0FdPqQCAFGzhPLp8zsJarW%2F%2F8KXxHsGDI0WPPnFz2jFSKDc7o6pkLDavJyAVhBQjg%3D%3D',
'pvRKIPC0dicOxZ%2FvMNIO%2FpvuiLdq5B9ugO3XnBVBBthwwZuD20IS19afpNhX94MPX1pvE4rGKewrbbvXqO6yeQ%3D%3D',
'zDf1VAbwhMoKbU3Tn%2FpUK5S7H7c3RdOU%2B%2FpReOEFBw24pImKGWejUvXCTJebWwXfp%2Byj95i7mF0oIFDFc5shfA%3D%3D',
'LItL5w1RWPukBFoiKQ%2FD7PpMoiPbmUdbIqzEHCMFaTuCWjul4w1D52XgsX0V58ZedNFdmOKFtiPT%2FWEHdiT5uw%3D%3D',
'Gd%2BMq2gtJ5b9dmlPh6r6hg0j6zXCN9dndZd%2FM3S7D0TqyXlRV%2FEyOX54yF7GTK4jms31lMokRk4vZy48ULCr6A%3D%3D',
'4fxQgZUy2hbO%2FEU61rpuNySpc3%2BGWZVXnRw0lkd0wireOAdSTyLaiErL%2FVOBnP5YTe5nXRJdFv3FyMcH%2Bh6lbg%3D%3D',
'%2FGN%2BQBieLa7gj0y5bGz9N7%2B%2BYY5J3T2VxMQ9K7nOF8mwLAabgyvcco7fYBW5o%2BHUIs98srxPiD%2FylJ0h5JqYtg%3D%3D',
'ebhoaoKgsYhoOWNmEmqqaPWV9rKLHJKfa6FCMfBFOfOh4%2B%2FEqp3P0GxwYxSdd6wHklE0OaNxrR0BPCLMgO4GWQ%3D%3D', 
'zzIkK5JvK5Qf0PQlG%2BGen30%2B74nnNDH%2BHUa8Gj21q0astwZNWgcy8Ul7O34wKGYUL3tgVelscNe0T1ngsr8LhA%3D%3D',
'T7e%2BPYf146tuYKxEcZDTv9CowyQ5IG6bt0vaig3oBkGyMYzOMMj43DA7NS2Kg6paIZr46vrlAANG0MYfSAG6ug%3D%3D', 
'A6hnPJg5aBy29z%2BQw%2B7O5qsGHwriJTM2tkdTnWqRK2GsBXBi7nPEg%2BlgO2kxTVbuEH5GWCijnDlmFYrjYtxJ8A%3D%3D',
'JQOuA6H%2B5GED2ebN1y3EeDRkQwr9BQczLJBUPE%2FHnD13IJJiWXLtRYd5PEtl%2FBqBUfojSzwyYk8SCvdIVcmYGQ%3D%3D',
'mhHYcau5uvPAk485nkwUxwPmSwHBOzgAvN5M8WYMsRLTBhbV8plJSMQcTXOJ4qelyWqJZZjLbSbx9hLDrt8GGg%3D%3D',
'9JY1jEJr%2Fo411aMc1WQhEBTR4EL4oEXVEuzo8d8GRMIuLJ9TJwJPj17%2FT%2Fl1i7nVvr%2FM7QDWtGF%2BOukWBs7T6w%3D%3D',
'3XgU8wCs8EYBaH%2BNNfp5FYhtq0Pmm8XbszveY0hANeC35UFCP2R4bblptP14vxU0MLXaMbeicjYca3GTni%2BwXA%3D%3D',
'jinBdygjITTh%2BdbEWG6DajHBUwB2BJbdjVGCpqi3HCAgmqgvZibEVIIz%2BqJ9j0isgQ6MH2IOgoqV5a82x83gZA%3D%3D',
'tCdOLU5Uzt%2FO0KmLIYqZSvUVXMFI09J9Ypf%2FR6zRz7Brg78PyFngcRRiz2%2FLIhmxSir4sJ5fMTlOTsl7kHYrug%3D%3D',
'dn0AsDO%2BDYT4eyDFEIsGiA518Hs%2B%2BKrEeSF9ZbrpYf6A3A2ySnnOCVJBu27ITk%2BdciNwaag4ULUcbSPNZvX3CA%3D%3D',
'xwgODeBQguCeaAZYpZRFwGjBiQJSBlq8XZsN86fBqnZ7msUod8uUfYtjCCDI7HlnpzPHnVlCbN61XHyMmh0VBA%3D%3D',
'liLwK4N1T04lqoN9rQKNX5UniCucaNFEWdDSjJL8lRs7veBVNLBOUczyMNNhVm1prmU4mZFmigmuOiNX9%2B48yg%3D%3D',
'4ehDKWXvirSm7MiRPEeFefnxApsVNXaw3WgxnOoLsOUIXUmZW6hHIFbgmkaYOUxK915rN1cTE6FbdkXkml8zjA%3D%3D',
'dNZWlZbeIWGzxUJH0qrd6Tgza4CjxdpdSUvbVsbWDsv1FkD1dV7J4ziLqsex40vkVGupL4VaKqqC9cFRM2mBsA%3D%3D',
'1famqleYz%2FAt4bgyjC2qMqqgnJJwYEmmMVYpJ06aoiKQnt1KEgTNQ6yW1lAs2QvIRsc3iT8IU7k9b32%2F8LyD4w%3D%3D',
"ifKRU73xKAdgp5Z2cQ6BDZSHhEOvKL8TNiEegFTFxtM9AWL6l0Wx8WJ4DfmQDZrTjjb509tqW0hO2wA9FE%2BWsQ%3D%3D"
]

In [6]:
len(serviceKey)

31

하나의 키로 하나의 버스 노선의 현재 운행중인 버스 목록을 가져오는 함수

In [7]:
def getData(key, routeID):
    url =  "http://ws.bus.go.kr/api/rest/buspos/getLowBusPosByRtid"
    data = requests.get(url+ "?serviceKey="+key+"&busRouteId="+routeID)
    xml = BeautifulSoup(data.text, "lxml")
    busList = xml.findAll("itemlist") #해당 노선의 현재 운행중인 모든 저상버스
    
    return busList


In [8]:
lowbusID = ['100100570', '100100549', '100100006', '100100007', '100100010', '100100012', '100100016', '100100015', '100100017', '100100583', '100100018', '100100019', '100100020', '100100021', '100100022', '100100023', '100100024', '100100025', '100100026', '100100027', '100100029', '100100030', '100100031', '100100032', '100100033', '100100034', '100100036', '110000002', '100100038', '100100039', '100100407', '100100042', '100100043', '100100044', '100100046', '100100047', '100100048', '100100049', '100100051', '100100052', '100100053', '100100564', '100100496', '100100055', '100100056', '124000038', '100100553', '100100057', '124000036', '100100061', '100100596', '100100062', '100100063', '100100597', '100100064', '100100461', '100100068', '100100409', '100100459', '100100070', '113000002', '100100071', '100100605', '100100073', '100100074', '100100075', '100100076', '100100077', '100100410', '100100078', '100100079', '100100080', '100100081', '100100082', '100100083', '100100574', '100100084', '100100085', '100100086', '100100087', '100100088', '100100090', '100100091', '100100093', '100100094', '100100096', '100100097', '100100098', '100100497', '115000007', '100100102', '100100550', '100100436', '100100495', '100100551', '100100440', '100100103', '100100104', '100100105', '100100107', '100100587', '100100511', '100100110', '100100111', '100100112', '100100537', '100100114', '100100565', '100100116', '100100117', '100100118', '111000014', '100100437', '111000011', '100100131', '100100143', '100100144', '100100148', '100100154', '100100155', '100100159', '100100164', '100100165', '100100175', '100100177', '100100179', '100100181', '100100184', '100100185', '100100186', '100100188', '100100189', '100100190', '100100522', '100100198', '100100199', '100100202', '100100203', '100100204', '100100205', '100100599', '100100611', '100100209', '100100210', '100100211', '100100438', '100100579', '100100213', '100100215', '100100216', '100100218', '100100220', '100100221', '100100222', '100100478', '100100498', '100100501', '100100578', '100100613', '100100223', '100100224', '100100225', '100100226', '100100228', '100100229', '100100232', '100100609', '100100612', '100100234', '100100427', '100100500', '100100454', '100100246', '122000001', '100100248', '100100254', '100100257', '100100258', '100100259', '100100260', '100100261', '100100263', '100100264', '100100265', '100100266', '100100267', '100100268', '100100269', '100100272', '100100273', '100100274', '100100275', '100100276', '100100277', '100100279', '100100282', '100100283', '100100284', '100100285', '100100288', '100100289', '100100290', '100100291', '100100292', '100100294', '100100521', '100100576', '100100300', '100100301', '100100304', '100100305', '100100306', '100100307', '100100308', '100100311', '100100313', '100100601', '100100602', '100100318', '115000005', '100100329', '100100330', '100100331', '100100332', '100100451', '100100449', '100100448', '100100447', '100100337', '100100338', '100100339', '100100340', '100100341', '100100446', '100100344', '100100499', '100100345', '100100346', '100100347', '100100348', '100100462', '100100352', '100100353', '100100360', '111000010', '100100001', '100100003', '165000381', '222000032', '207000004', '213000011', '222000045', '235000020', '234000040', '210000002', '229000041', '222000028', '222000048', '222000058', '232000095', '227000006', '227000001', '234000003', '207000027', '210000013', '210000003', '210000001', '204000012', '224000008', '210000009', '204000005', '210000069', '210000042', '204000013', '210000037', '207000002', '236000048', '233000031', '200000149', '210000093', '222000084', '212000002', '222000008', '222000111', '222000027', '222000009']

In [9]:
len(lowbusID)

286

각각의 버스노선을 객체로 만들어 현재 운행중인 해당 노선의 버스 목록과 각 버스의 위치정보 제공시간을 저장함

In [10]:
class LowBus:
    def __init__(self, routeID):
        self.id = routeID
        self.busList = {}
        self.bus_time = {}

In [11]:
lowbusList= []

for lowbus in lowbusID:
    lowbusList.append(LowBus(lowbus))   #전체 lowbus 객체 생성

In [12]:
import datetime as dt

5시~1시까지 반복

In [13]:
while True:
    today = dt.datetime.now().day
    now = dt.datetime.now().hour
    
    if now >=5 or now < 1:
    
        for key in serviceKey:
            
            if dt.datetime.now().day > today and dt.datetime.now().hour >= 1:
                    break
                
            no_bus = False

            call =  10000 // len(lowbusID)
#             print(key)
        #     call = 4
            while call > 0 :  #call 수 만큼 반복 가능 - 할당된 call을 다 사용하면 다음 서비스키 사용
                
                if dt.datetime.now().day > today and dt.datetime.now().hour >= 1:
                    break
                         
                no_bus = True

                for lowbus in lowbusList:
#                     print(lowbus.id)
                    lowbus.busList = getData(key, lowbus.id)
                    call -= 1


                    if lowbus.busList:
                        no_bus = False
                        for bus in lowbus.busList:
                            vehid = bus.vehid.text
                            stop = int(bus.stopflag.text)
                            bus_time = bus.datatm.text[8:]
#                             print('vehid: ', vehid)
#                             print('stop: ', stop)
#                             print('time: ', bus_time)
#                             print('x, y: ',bus.tmx.text, ', ',bus.tmy.text)

                            if vehid in lowbus.bus_time.keys():
                                if lowbus.bus_time[vehid]!=bus_time and stop:  #정류장 도착
                                    #DB에 시간, 위치 저장
                                    conn.execute(Timetable.insert(), routeId = lowbus.id, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
#                                     print('db update')
                            elif stop == 1:
                                    #DB에 시간 저장
                                    conn.execute(Timetable.insert(), routeId = lowbus.id, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
#                                     print('db update')
                            lowbus.bus_time[vehid] = bus_time

                            if vehid in lowbus.bus_time.keys() and (int(bus.nextsttm.text) >= int(bus.laststtm.text)):  #종점에 도착한 버스는 pop
                                lowbus.bus_time.pop(vehid)

#                             print()

                    if call <= 0:
                        break

#                     print("-----------------------------")      
                    

                time.sleep(60)       

            if no_bus:
#                 print('no bus')
                break
    else:
        time.sleep(900)

ConnectionError: HTTPConnectionPool(host='ws.bus.go.kr', port=80): Max retries exceeded with url: /api/rest/buspos/getLowBusPosByRtid?serviceKey=9JY1jEJr%2Fo411aMc1WQhEBTR4EL4oEXVEuzo8d8GRMIuLJ9TJwJPj17%2FT%2Fl1i7nVvr%2FM7QDWtGF%2BOukWBs7T6w%3D%3D&busRouteId=100100570 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000164662335C0>: Failed to establish a new connection: [WinError 10053] 현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다',))

In [ ]:
for row in conn.execute(select([Timetable])):
    print(row)

In [ ]:
dt.datetime.now().day

In [1]:
import pandas